# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


In [60]:
class Matrix:
    def __init__(self, *args):
        if len(args) == 2:  # Initialize with n (rows) and m (columns)
            self.rows = args[0]
            self.cols = args[1]
            self.data = [[0] * self.cols for _ in range(self.rows)]
        elif len(args) == 1 and isinstance(args[0], list):
            self.data = args[0]
            self.rows = len(self.data)
            self.cols = max(len(row) for row in self.data) if self.rows > 0 else 0
            for i in range(self.rows):
                while len(self.data[i]) < self.cols:
                    self.data[i].append(0)
        else:
            raise ValueError("Invalid arguments. Use either (n, m) or a list of lists.")

    def __repr__(self):
        return f"Matrix({self.data})"

    def __str__(self):
        return "\n".join(" ".join(map(str, row)) for row in self.data)

    # Indexing with M[i][j] and M[i, j]
    def __getitem__(self, key):
        if isinstance(key, tuple):  # For M[i, j] type indexing
            i, j = key
            if 0 <= i < self.rows and 0 <= j < self.cols:
                return self.data[i][j]
            raise IndexError("Index out of range")
        elif isinstance(key, int):  # For M[i] type indexing (returns the i-th row)
            if 0 <= key < self.rows:
                return self.data[key]
            raise IndexError("Index out of range")
        else:
            raise TypeError("Invalid index type")

    def __setitem__(self, key, value):
        if isinstance(key, tuple):  # For M[i, j] type indexing
            i, j = key
            if 0 <= i < self.rows and 0 <= j < self.cols:
                self.data[i][j] = value
            else:
                raise IndexError("Index out of range")
        elif isinstance(key, int):  # For M[i] type indexing (setting the entire row)
            if 0 <= key < self.rows:
                if isinstance(value, list) and len(value) == self.cols:
                    self.data[key] = value
                else:
                    raise ValueError("Row must be a list with the correct number of columns")
            else:
                raise IndexError("Index out of range")
        else:
            raise TypeError("Invalid index type")

    def __assign__(self, other):
        # Matrix assignment: M1 = M2
        if isinstance(other, Matrix):
            if self.rows == other.rows and self.cols == other.cols:
                self.data = [row[:] for row in other.data]  # Deep copy of the data
            else:
                raise ValueError("Matrices must have the same size for assignment.")
        elif isinstance(other, list) and len(other) == self.rows and len(other[0]) == self.cols:
            self.data = [row[:] for row in other]  # Deep copy from list of lists
        else:
            raise ValueError("Invalid assignment. List of lists must have the correct size.")

# Example usage:
# Initialize with a jagged list of lists
matrix1 = Matrix([[1, 2], [3, 4, 5], [6]])
print(f"Matrix with uneven columns:\n{matrix1}\n")

# Initialize with dimensions
matrix2 = Matrix(3, 4)
print(f"Initialize matrix with dimenstions:\n{matrix2}\n")

# Set and get values using indexing M[i, j] and M[i][j]
matrix2[1, 1] = 5
print(f"Set and get index of matrix:\n{matrix2[1, 1]}\n")  # Should print 5

matrix2[0] = [1, 2, 3, 4]  # Set first row
print(f"Set first row of matrix:\n{matrix2}\n")

# Assign one matrix to another
matrix3 = Matrix([[10, 20, 30], [40, 50, 60], [70, 80, 90]])
matrix4 = Matrix(3, 3)
matrix4.__assign__(matrix3)
print(f"Assign matrix to another:\n{matrix4}")

# Trying invalid assignment
# matrix4.__assign__(Matrix(2, 3))  # This should raise a ValueError due to size mismatch


Matrix with uneven columns:
1 2 0
3 4 5
6 0 0

Initialize matrix with dimenstions:
0 0 0 0
0 0 0 0
0 0 0 0

Set and get index of matrix:
5

Set first row of matrix:
1 2 3 4
0 5 0 0
0 0 0 0

Assign matrix to another:
10 20 30
40 50 60
70 80 90


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

In [50]:
class Matrix:
    def __init__(self, *args):
        if len(args) == 2:  # Initialize with n (rows) and m (columns)
            self.rows = args[0]
            self.cols = args[1]
            self.data = [[0] * self.cols for _ in range(self.rows)]
        elif len(args) == 1 and isinstance(args[0], list):
            self.data = args[0]
            self.rows = len(self.data)
            self.cols = max(len(row) for row in self.data) if self.rows > 0 else 0
            for i in range(self.rows):
                while len(self.data[i]) < self.cols:
                    self.data[i].append(0)
        else:
            raise ValueError("Invalid arguments. Use either (n, m) or a list of lists.")

    def __repr__(self):
        return f"Matrix({self.data})"

    def __str__(self):
        return "\n".join(" ".join(map(str, row)) for row in self.data)

    # Indexing with M[i][j] and M[i, j]
    def __getitem__(self, key):
        if isinstance(key, tuple):  # For M[i, j] type indexing
            i, j = key
            if 0 <= i < self.rows and 0 <= j < self.cols:
                return self.data[i][j]
            raise IndexError("Index out of range")
        elif isinstance(key, int):  # For M[i] type indexing (returns the i-th row)
            if 0 <= key < self.rows:
                return self.data[key]
            raise IndexError("Index out of range")
        else:
            raise TypeError("Invalid index type")

    def __setitem__(self, key, value):
        if isinstance(key, tuple):  # For M[i, j] type indexing
            i, j = key
            if 0 <= i < self.rows and 0 <= j < self.cols:
                self.data[i][j] = value
            else:
                raise IndexError("Index out of range")
        elif isinstance(key, int):  # For M[i] type indexing (setting the entire row)
            if 0 <= key < self.rows:
                if isinstance(value, list) and len(value) == self.cols:
                    self.data[key] = value
                else:
                    raise ValueError("Row must be a list with the correct number of columns")
            else:
                raise IndexError("Index out of range")
        else:
            raise TypeError("Invalid index type")

    def __assign__(self, other):
        # Matrix assignment: M1 = M2
        if isinstance(other, Matrix):
            if self.rows == other.rows and self.cols == other.cols:
                self.data = [row[:] for row in other.data]  # Deep copy of the data
            else:
                raise ValueError("Matrices must have the same size for assignment.")
        elif isinstance(other, list) and len(other) == self.rows and len(other[0]) == self.cols:
            self.data = [row[:] for row in other]  # Deep copy from list of lists
        else:
            raise ValueError("Invalid assignment. List of lists must have the correct size.")
    
    def shape(self):
        # Return (col, row) shape
        return (self.cols, self.rows)
    
    def transpose(self):
        # Create the transpose by flipping rows and columns
        transposed_data = [[self.data[i][j] for i in range(self.rows)] for j in range(self.cols)]
        return Matrix(transposed_data)
    
    def row(self, n):
        # Return row as a matrix of nth row of matrix data
        row_data = [[self.data[n-1][i] for i in range(self.cols)]]
        return Matrix(row_data)
        
    def column(self, n):
        # Return column as a matrix of nth column of matrix data
        column_data = [[self.data[i][n-1] for i in range(self.rows)]]
        return Matrix(column_data)
    
    def to_list(self):
        # Return matrix as list of lists
        listOfMatrix = [[self.data[i][j] for j in range(self.cols)] for i in range(self.rows)]
        return listOfMatrix
    
    def block(self, n_0, n_1, m_0, m_1):
        # Return block of matrix where n is the columns and m is the rows
        block_data = [[self.data[i][j] for j in range(n_0-1, n_1)] for i in range(m_0-1, m_1)]
        return Matrix(block_data)

# Example usage:
# Return original matrix
matrix1 = Matrix([[10, 20, 30], [40, 50, 60], [70, 80, 90]])
print(f"Original Matrix:\n{matrix1}\n")

# Return shape of matrix
print(f"Shape of Matrix:\n{matrix1.shape()}\n")

# Return transpose of matrix
print(f"Transpose of Matrix:\n{matrix1.transpose()}\n")

# Return nth row of matrix
print(f"Row: {matrix1.row(2)}\n")

# Return nth column of matrix
print(f"Column: {matrix1.column(2)}\n")

# Return matrix as a list of lists
print(f"Matrix as list of lists: {matrix1.to_list()}\n")

# Return block of the matrix
print(f"Block of matrix:\n{matrix1.block(2, 3, 2, 3)}\n")

Original Matrix:
10 20 30
40 50 60
70 80 90

Shape of Matrix:
(3, 3)

Transpose of Matrix:
10 40 70
20 50 80
30 60 90

Row: 40 50 60

Column: 20 50 80

Matrix as list of lists: [[10, 20, 30], [40, 50, 60], [70, 80, 90]]

Block of matrix:
50 60
80 90



3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

In [51]:
def constant(n, m, c):
    mat = [[c for j in range(m)] for i in range(n)]
    return Matrix(mat)

def zeros(n, m):
    return constant(n, m, 0.0)

def ones(n, m):
    return constant(n, m, 1.0)

def eye(n):
    return Matrix([[1 if i == j else 0 for j in range(n)] for i in range(n)])

print(f"{constant(3, 4, 10)}\n")
print(f"{zeros(3, 4)}\n")
print(f"{ones(3, 4)}\n")
print(f"{eye(3)}\n")

10 10 10 10
10 10 10 10
10 10 10 10

0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0
0.0 0.0 0.0 0.0

1.0 1.0 1.0 1.0
1.0 1.0 1.0 1.0
1.0 1.0 1.0 1.0

1 0 0
0 1 0
0 0 1



4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

In [52]:
class Matrix:
    def __init__(self, *args):
        if len(args) == 2:  # Initialize with n (rows) and m (columns)
            self.rows = args[0]
            self.cols = args[1]
            self.data = [[0] * self.cols for _ in range(self.rows)]
        elif len(args) == 1 and isinstance(args[0], list):
            self.data = args[0]
            self.rows = len(self.data)
            self.cols = max(len(row) for row in self.data) if self.rows > 0 else 0
            for i in range(self.rows):
                while len(self.data[i]) < self.cols:
                    self.data[i].append(0)
        else:
            raise ValueError("Invalid arguments. Use either (n, m) or a list of lists.")

    def __repr__(self):
        return f"Matrix({self.data})"

    def __str__(self):
        return "\n".join(" ".join(map(str, row)) for row in self.data)

    # Indexing with M[i][j] and M[i, j]
    def __getitem__(self, key):
        if isinstance(key, tuple):  # For M[i, j] type indexing
            i, j = key
            if 0 <= i < self.rows and 0 <= j < self.cols:
                return self.data[i][j]
            raise IndexError("Index out of range")
        elif isinstance(key, int):  # For M[i] type indexing (returns the i-th row)
            if 0 <= key < self.rows:
                return self.data[key]
            raise IndexError("Index out of range")
        else:
            raise TypeError("Invalid index type")

    def __setitem__(self, key, value):
        if isinstance(key, tuple):  # For M[i, j] type indexing
            i, j = key
            if 0 <= i < self.rows and 0 <= j < self.cols:
                self.data[i][j] = value
            else:
                raise IndexError("Index out of range")
        elif isinstance(key, int):  # For M[i] type indexing (setting the entire row)
            if 0 <= key < self.rows:
                if isinstance(value, list) and len(value) == self.cols:
                    self.data[key] = value
                else:
                    raise ValueError("Row must be a list with the correct number of columns")
            else:
                raise IndexError("Index out of range")
        else:
            raise TypeError("Invalid index type")

    def __assign__(self, other):
        # Matrix assignment: M1 = M2
        if isinstance(other, Matrix):
            if self.rows == other.rows and self.cols == other.cols:
                self.data = [row[:] for row in other.data]  # Deep copy of the data
            else:
                raise ValueError("Matrices must have the same size for assignment.")
        elif isinstance(other, list) and len(other) == self.rows and len(other[0]) == self.cols:
            self.data = [row[:] for row in other]  # Deep copy from list of lists
        else:
            raise ValueError("Invalid assignment. List of lists must have the correct size.")
    
    def shape(self):
        # Return (col, row) shape
        return (self.cols, self.rows)
    
    def transpose(self):
        # Create the transpose by flipping rows and columns
        transposed_data = [[self.data[i][j] for i in range(self.rows)] for j in range(self.cols)]
        return Matrix(transposed_data)
    
    def row(self, n):
        # Return row as a matrix of nth row of matrix data
        row_data = [[self.data[n-1][i] for i in range(self.cols)]]
        return Matrix(row_data)
        
    def column(self, n):
        # Return column as a matrix of nth column of matrix data
        column_data = [[self.data[i][n-1] for i in range(self.rows)]]
        return Matrix(column_data)
    
    def to_list(self):
        # Return matrix as list of lists
        listOfMatrix = [[self.data[i][j] for j in range(self.cols)] for i in range(self.rows)]
        return listOfMatrix
    
    def block(self, n_0, n_1, m_0, m_1):
        # Return block of matrix where n is the columns and m is the rows
        block_data = [[self.data[i][j] for j in range(n_0-1, n_1)] for i in range(m_0-1, m_1)]
        return Matrix(block_data)
    
    
    # Matrix Operations
    def scalarmul(self, c):
        # Returns a matrix that is the scalar product of c * M
        result = [[c * self.data[i][j] for j in range(self.cols)] for i in range(self.rows)]
        return Matrix(result)

    def add(self, N):
        # Adds two matrices
        if self.rows != N.rows or self.cols != N.cols:
            raise ValueError("Matrices must have the same dimensions for addition.")
        result = [[self.data[i][j] + N.data[i][j] for j in range(self.cols)] for i in range(self.rows)]
        return Matrix(result)

    def sub(self, N):
        # Subtracts two matrices
        if self.rows != N.rows or self.cols != N.cols:
            raise ValueError("Matrices must have the same dimensions for subtraction.")
        result = [[self.data[i][j] - N.data[i][j] for j in range(self.cols)] for i in range(self.rows)]
        return Matrix(result)

    def mat_mult(self, N):
        # Matrix multiplication: Returns the matrix product of two matrices
        if self.cols != N.rows:
            raise ValueError("Number of columns of the first matrix must match the number of rows of the second matrix.")
        result = [[sum(self.data[i][k] * N.data[k][j] for k in range(self.cols)) for j in range(N.cols)] for i in range(self.rows)]
        return Matrix(result)

    def element_mult(self, N):
        # Element-wise multiplication: Returns a matrix with the element-wise product of two matrices
        if self.rows != N.rows or self.cols != N.cols:
            raise ValueError("Matrices must have the same dimensions for element-wise multiplication.")
        result = [[self.data[i][j] * N.data[i][j] for j in range(self.cols)] for i in range(self.rows)]
        return Matrix(result)

    def equals(self, N):
        # Returns True if two matrices are equal, False otherwise
        return self == N

# Example usage:

# Create two matrices
A = Matrix([[1, 2], [3, 4]])
B = Matrix([[5, 6], [7, 8]])

# Return matrix A and B
print(f"Matrix A:\n{A}\n")
print(f"Matrix B:\n{B}\n")

# Scalar multiplication
C = A.scalarmul(2)
print("Scalar multiplication of A * 2:")
print(C)

# Matrix addition
D = A.add(B)
print("\nMatrix A + B:")
print(D)

# Matrix subtraction
E = A.sub(B)
print("\nMatrix A - B:")
print(E)

# Matrix multiplication
F = A.mat_mult(B)
print("\nMatrix A * B (Matrix multiplication):")
print(F)

# Element-wise multiplication
G = A.element_mult(B)
print("\nMatrix A element-wise * B:")
print(G)

# Matrix equality
print("\nIs matrix A equal to B?")
print(A.equals(B))  # Should return False

Matrix A:
1 2
3 4

Matrix B:
5 6
7 8

Scalar multiplication of A * 2:
2 4
6 8

Matrix A + B:
6 8
10 12

Matrix A - B:
-4 -4
-4 -4

Matrix A * B (Matrix multiplication):
19 22
43 50

Matrix A element-wise * B:
5 12
21 32

Is matrix A equal to B?
False


5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


In [53]:
class Matrix:
    def __init__(self, *args):
        if len(args) == 2:  # Initialize with n (rows) and m (columns)
            self.rows = args[0]
            self.cols = args[1]
            self.data = [[0] * self.cols for _ in range(self.rows)]
        elif len(args) == 1 and isinstance(args[0], list):
            self.data = args[0]
            self.rows = len(self.data)
            self.cols = max(len(row) for row in self.data) if self.rows > 0 else 0
            for i in range(self.rows):
                while len(self.data[i]) < self.cols:
                    self.data[i].append(0)
        else:
            raise ValueError("Invalid arguments. Use either (n, m) or a list of lists.")

    def __repr__(self):
        return f"Matrix({self.data})"

    def __str__(self):
        return "\n".join(" ".join(map(str, row)) for row in self.data)

    # Indexing with M[i][j] and M[i, j]
    def __getitem__(self, key):
        if isinstance(key, tuple):  # For M[i, j] type indexing
            i, j = key
            if 0 <= i < self.rows and 0 <= j < self.cols:
                return self.data[i][j]
            raise IndexError("Index out of range")
        elif isinstance(key, int):  # For M[i] type indexing (returns the i-th row)
            if 0 <= key < self.rows:
                return self.data[key]
            raise IndexError("Index out of range")
        else:
            raise TypeError("Invalid index type")

    def __setitem__(self, key, value):
        if isinstance(key, tuple):  # For M[i, j] type indexing
            i, j = key
            if 0 <= i < self.rows and 0 <= j < self.cols:
                self.data[i][j] = value
            else:
                raise IndexError("Index out of range")
        elif isinstance(key, int):  # For M[i] type indexing (setting the entire row)
            if 0 <= key < self.rows:
                if isinstance(value, list) and len(value) == self.cols:
                    self.data[key] = value
                else:
                    raise ValueError("Row must be a list with the correct number of columns")
            else:
                raise IndexError("Index out of range")
        else:
            raise TypeError("Invalid index type")
    
    def __assign__(self, other):
        # Matrix assignment: M1 = M2
        if isinstance(other, Matrix):
            if self.rows == other.rows and self.cols == other.cols:
                self.data = [row[:] for row in other.data]  # Deep copy of the data
            else:
                raise ValueError("Matrices must have the same size for assignment.")
        elif isinstance(other, list) and len(other) == self.rows and len(other[0]) == self.cols:
            self.data = [row[:] for row in other]  # Deep copy from list of lists
        else:
            raise ValueError("Invalid assignment. List of lists must have the correct size.")
    
    def shape(self):
        # Return (col, row) shape
        return (self.cols, self.rows)
    
    def transpose(self):
        # Create the transpose by flipping rows and columns
        transposed_data = [[self.data[i][j] for i in range(self.rows)] for j in range(self.cols)]
        return Matrix(transposed_data)
    
    def row(self, n):
        # Return row as a matrix of nth row of matrix data
        row_data = [[self.data[n-1][i] for i in range(self.cols)]]
        return Matrix(row_data)
        
    def column(self, n):
        # Return column as a matrix of nth column of matrix data
        column_data = [[self.data[i][n-1] for i in range(self.rows)]]
        return Matrix(column_data)
    
    def to_list(self):
        # Return matrix as list of lists
        listOfMatrix = [[self.data[i][j] for j in range(self.cols)] for i in range(self.rows)]
        return listOfMatrix
    
    def block(self, n_0, n_1, m_0, m_1):
        # Return block of matrix where n is the columns and m is the rows
        block_data = [[self.data[i][j] for j in range(n_0-1, n_1)] for i in range(m_0-1, m_1)]
        return Matrix(block_data)
    
    # Matrix Operations
    def scalarmul(self, c):
        # Returns a matrix that is the scalar product of c * M
        result = [[c * self.data[i][j] for j in range(self.cols)] for i in range(self.rows)]
        return Matrix(result)

    def add(self, N):
        # Adds two matrices
        if self.rows != N.rows or self.cols != N.cols:
            raise ValueError("Matrices must have the same dimensions for addition.")
        result = [[self.data[i][j] + N.data[i][j] for j in range(self.cols)] for i in range(self.rows)]
        return Matrix(result)

    def sub(self, N):
        # Subtracts two matrices
        if self.rows != N.rows or self.cols != N.cols:
            raise ValueError("Matrices must have the same dimensions for subtraction.")
        result = [[self.data[i][j] - N.data[i][j] for j in range(self.cols)] for i in range(self.rows)]
        return Matrix(result)

    def mat_mult(self, N):
        # Matrix multiplication: Returns the matrix product of two matrices
        if self.cols != N.rows:
            raise ValueError("Number of columns of the first matrix must match the number of rows of the second matrix.")
        result = [[sum(self.data[i][k] * N.data[k][j] for k in range(self.cols)) for j in range(N.cols)] for i in range(self.rows)]
        return Matrix(result)

    def element_mult(self, N):
        # Element-wise multiplication: Returns a matrix with the element-wise product of two matrices
        if self.rows != N.rows or self.cols != N.cols:
            raise ValueError("Matrices must have the same dimensions for element-wise multiplication.")
        result = [[self.data[i][j] * N.data[i][j] for j in range(self.cols)] for i in range(self.rows)]
        return Matrix(result)

    def equals(self, N):
        # Returns True if two matrices are equal, False otherwise
        return self == N

    # Operator overloading
    def __mul__(self, other):
        # Handles scalar multiplication and matrix multiplication
        if isinstance(other, (int, float)):  # Scalar multiplication (M * c)
            return self.scalarmul(other)
        elif isinstance(other, Matrix):  # Matrix multiplication (M * N)
            return self.mat_mult(other)
        else:
            raise TypeError("Unsupported multiplication type.")

    def __rmul__(self, other):
        # Handles scalar multiplication when the scalar is on the left (c * M)
        if isinstance(other, (int, float)):
            return self.scalarmul(other)
        else:
            raise TypeError("Unsupported multiplication type.")

    def __add__(self, other):
        # Matrix addition
        if isinstance(other, Matrix):
            return self.add(other)
        else:
            raise TypeError("Unsupported addition type.")

    def __sub__(self, other):
        # Matrix subtraction
        if isinstance(other, Matrix):
            return self.sub(other)
        else:
            raise TypeError("Unsupported subtraction type.")

    def __eq__(self, other):
        # Matrix equality check
        if isinstance(other, Matrix):
            return self.data == other.data
        return False

# Create matrices and demonstrate operator overloading
M = Matrix([[1, 2], [3, 4]])
N = Matrix([[5, 6], [7, 8]])

# Return matrix A and B
print(f"Matrix M:\n{M}\n")
print(f"Matrix N:\n{N}\n")

# Scalar multiplication (2 * M)
result = 2 * M
print("2 * M:")
print(result)

# Scalar multiplication (M * 2)
result = M * 2
print("\nM * 2:")
print(result)

# Matrix addition (M + N)
result = M + N
print("\nM + N:")
print(result)

# Matrix subtraction (M - N)
result = M - N
print("\nM - N:")
print(result)

# Matrix multiplication (M * N)
result = M * N
print("\nM * N:")
print(result)

# Matrix equality check (M == N)
print("\nM == N:", M == N)

# Matrix assignment (M = N)
M = N
print("\nM after assignment to N:")
print(M)

Matrix M:
1 2
3 4

Matrix N:
5 6
7 8

2 * M:
2 4
6 8

M * 2:
2 4
6 8

M + N:
6 8
10 12

M - N:
-4 -4
-4 -4

M * N:
19 22
43 50

M == N: False

M after assignment to N:
5 6
7 8


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [68]:
class Matrix:
    def __init__(self, *args):
        if len(args) == 2:  # Initialize with n (rows) and m (columns)
            self.rows = args[0]
            self.cols = args[1]
            self.data = [[0] * self.cols for _ in range(self.rows)]
        elif len(args) == 1 and isinstance(args[0], list):
            self.data = args[0]
            self.rows = len(self.data)
            self.cols = max(len(row) for row in self.data) if self.rows > 0 else 0
            for i in range(self.rows):
                while len(self.data[i]) < self.cols:
                    self.data[i].append(0)
        else:
            raise ValueError("Invalid arguments. Use either (n, m) or a list of lists.")

    def __repr__(self):
        return f"Matrix({self.data})"

    def __str__(self):
        return "\n".join(" ".join(map(str, row)) for row in self.data)

    # Indexing with M[i][j] and M[i, j]
    def __getitem__(self, key):
        if isinstance(key, tuple):  # For M[i, j] type indexing
            i, j = key
            if 0 <= i < self.rows and 0 <= j < self.cols:
                return self.data[i][j]
            raise IndexError("Index out of range")
        elif isinstance(key, int):  # For M[i] type indexing (returns the i-th row)
            if 0 <= key < self.rows:
                return self.data[key]
            raise IndexError("Index out of range")
        else:
            raise TypeError("Invalid index type")

    def __setitem__(self, key, value):
        if isinstance(key, tuple):  # For M[i, j] type indexing
            i, j = key
            if 0 <= i < self.rows and 0 <= j < self.cols:
                self.data[i][j] = value
            else:
                raise IndexError("Index out of range")
        elif isinstance(key, int):  # For M[i] type indexing (setting the entire row)
            if 0 <= key < self.rows:
                if isinstance(value, list) and len(value) == self.cols:
                    self.data[key] = value
                else:
                    raise ValueError("Row must be a list with the correct number of columns")
            else:
                raise IndexError("Index out of range")
        else:
            raise TypeError("Invalid index type")
    
    def __assign__(self, other):
        # Matrix assignment: M1 = M2
        if isinstance(other, Matrix):
            if self.rows == other.rows and self.cols == other.cols:
                self.data = [row[:] for row in other.data]  # Deep copy of the data
            else:
                raise ValueError("Matrices must have the same size for assignment.")
        elif isinstance(other, list) and len(other) == self.rows and len(other[0]) == self.cols:
            self.data = [row[:] for row in other]  # Deep copy from list of lists
        else:
            raise ValueError("Invalid assignment. List of lists must have the correct size.")
    
    def shape(self):
        # Return (col, row) shape
        return (self.cols, self.rows)
    
    def transpose(self):
        # Create the transpose by flipping rows and columns
        transposed_data = [[self.data[i][j] for i in range(self.rows)] for j in range(self.cols)]
        return Matrix(transposed_data)
    
    def row(self, n):
        # Return row as a matrix of nth row of matrix data
        row_data = [[self.data[n-1][i] for i in range(self.cols)]]
        return Matrix(row_data)
        
    def column(self, n):
        # Return column as a matrix of nth column of matrix data
        column_data = [[self.data[i][n-1] for i in range(self.rows)]]
        return Matrix(column_data)
    
    def to_list(self):
        # Return matrix as list of lists
        listOfMatrix = [[self.data[i][j] for j in range(self.cols)] for i in range(self.rows)]
        return listOfMatrix
    
    def block(self, n_0, n_1, m_0, m_1):
        # Return block of matrix where n is the columns and m is the rows
        block_data = [[self.data[i][j] for j in range(n_0-1, n_1)] for i in range(m_0-1, m_1)]
        return Matrix(block_data)
    
    # Matrix Operations
    def scalarmul(self, c):
        # Returns a matrix that is the scalar product of c * M
        result = [[c * self.data[i][j] for j in range(self.cols)] for i in range(self.rows)]
        return Matrix(result)

    def add(self, N):
        # Adds two matrices
        if self.rows != N.rows or self.cols != N.cols:
            raise ValueError("Matrices must have the same dimensions for addition.")
        result = [[self.data[i][j] + N.data[i][j] for j in range(self.cols)] for i in range(self.rows)]
        return Matrix(result)

    def sub(self, N):
        # Subtracts two matrices
        if self.rows != N.rows or self.cols != N.cols:
            raise ValueError("Matrices must have the same dimensions for subtraction.")
        result = [[self.data[i][j] - N.data[i][j] for j in range(self.cols)] for i in range(self.rows)]
        return Matrix(result)

    def mat_mult(self, N):
        # Matrix multiplication: Returns the matrix product of two matrices
        if self.cols != N.rows:
            raise ValueError("Number of columns of the first matrix must match the number of rows of the second matrix.")
        result = [[sum(self.data[i][k] * N.data[k][j] for k in range(self.cols)) for j in range(N.cols)] for i in range(self.rows)]
        return Matrix(result)

    def element_mult(self, N):
        # Element-wise multiplication: Returns a matrix with the element-wise product of two matrices
        if self.rows != N.rows or self.cols != N.cols:
            raise ValueError("Matrices must have the same dimensions for element-wise multiplication.")
        result = [[self.data[i][j] * N.data[i][j] for j in range(self.cols)] for i in range(self.rows)]
        return Matrix(result)

    def equals(self, N):
        # Returns True if two matrices are equal, False otherwise
        return self == N

    # Operator overloading
    def __mul__(self, other):
        # Handles scalar multiplication and matrix multiplication
        if isinstance(other, (int, float)):  # Scalar multiplication (M * c)
            return self.scalarmul(other)
        elif isinstance(other, Matrix):  # Matrix multiplication (M * N)
            return self.mat_mult(other)
        else:
            raise TypeError("Unsupported multiplication type.")

    def __rmul__(self, other):
        # Handles scalar multiplication when the scalar is on the left (c * M)
        if isinstance(other, (int, float)):
            return self.scalarmul(other)
        else:
            raise TypeError("Unsupported multiplication type.")

    def __add__(self, other):
        # Matrix addition
        if isinstance(other, Matrix):
            return self.add(other)
        else:
            raise TypeError("Unsupported addition type.")

    def __sub__(self, other):
        # Matrix subtraction
        if isinstance(other, Matrix):
            return self.sub(other)
        else:
            raise TypeError("Unsupported subtraction type.")

    def __eq__(self, other):
        # Matrix equality check
        if isinstance(other, Matrix):
            return self.data == other.data
        return False

# Matrix class implementation goes here (from the previous code)
# Assuming the Matrix class is already defined as shown in previous messages.

# Create matrices A, B, and C
A = Matrix([[1, 2], [3, 4]])
B = Matrix([[5, 6], [7, 8]])
C = Matrix([[9, 10], [11, 12]])

# Identity matrix I (2x2)
I = Matrix([[1, 0], [0, 1]])

# Distributive property: (AB)C = A(BC)
AB = A * B
AB_C = AB * C
A_BC = A * (B * C)
print("(AB)C = A(BC):")
print("AB * C:")
print(AB_C)
print("\nA * (B * C):")
print(A_BC)
print(AB_C == A_BC)

print("\n")

# Distributive property: A(B + C) = AB + AC
B_plus_C = B + C
A_B_plus_C = A * B_plus_C
A_B = A * B
A_C = A * C
AB_plus_AC = A_B + A_C
print("A(B + C) = AB + AC:")
print("A * (B + C):")
print(A_B_plus_C)
print("\nAB + AC:")
print(AB_plus_AC)
print(A_B_plus_C == AB_plus_AC)

print("\n")

# Non-commutativity of matrix multiplication: AB != BA
BA = B * A
print("AB != BA:")
print("AB:")
print(AB)
print("\nBA:")
print(BA)
print(AB == BA)

print("\n")

# Identity matrix property: AI = A
AI = A * I
print("AI = A:")
print("A * I:")
print(AI)
print("\nA:")
print(A)
print(AI == A)


(AB)C = A(BC):
AB * C:
413 454
937 1030

A * (B * C):
413 454
937 1030
True


A(B + C) = AB + AC:
A * (B + C):
50 56
114 128

AB + AC:
50 56
114 128
True


AB != BA:
AB:
19 22
43 50

BA:
23 34
31 46
False


AI = A:
A * I:
1 2
3 4

A:
1 2
3 4
True


### Quiz 2 Functions:

Write a function `make_deck` that returns a list of all of the cards in a standard card deck. The return should be a list of tuples of pairs of suit and value. For example the 10 of Clubs would be ('Clubs', 10) and Queen of Hearts would be ('Hearts', 'Queen'). Recall that a deck has 52 cards, divided into 4 suits (Clubs, Diamonds, Hearts, and Spades), and that each suit has 13 cards: 2 to 10, Jack, Queen, King, and Ace.

In [69]:
def make_deck():
    suits = ["Hearts", "Diamonds", "Clubs", "Spades"]
    cards = ["Ace", "2", "3", "4", "5", "6", "7", "8", "9", "Jack", "Queen", "King"]
    deck = []
    
    for i in range(len(suits)):
        for j in range(len(cards)):
            deck.append((suits[i], cards[j]))
            
    return deck

print(make_deck())
        

[('Hearts', 'Ace'), ('Hearts', '2'), ('Hearts', '3'), ('Hearts', '4'), ('Hearts', '5'), ('Hearts', '6'), ('Hearts', '7'), ('Hearts', '8'), ('Hearts', '9'), ('Hearts', 'Jack'), ('Hearts', 'Queen'), ('Hearts', 'King'), ('Diamonds', 'Ace'), ('Diamonds', '2'), ('Diamonds', '3'), ('Diamonds', '4'), ('Diamonds', '5'), ('Diamonds', '6'), ('Diamonds', '7'), ('Diamonds', '8'), ('Diamonds', '9'), ('Diamonds', 'Jack'), ('Diamonds', 'Queen'), ('Diamonds', 'King'), ('Clubs', 'Ace'), ('Clubs', '2'), ('Clubs', '3'), ('Clubs', '4'), ('Clubs', '5'), ('Clubs', '6'), ('Clubs', '7'), ('Clubs', '8'), ('Clubs', '9'), ('Clubs', 'Jack'), ('Clubs', 'Queen'), ('Clubs', 'King'), ('Spades', 'Ace'), ('Spades', '2'), ('Spades', '3'), ('Spades', '4'), ('Spades', '5'), ('Spades', '6'), ('Spades', '7'), ('Spades', '8'), ('Spades', '9'), ('Spades', 'Jack'), ('Spades', 'Queen'), ('Spades', 'King')]
